In [1]:
#Effect Modification by Lifestyle Score
library(ggpubr)
library(dplyr)
library(mice)
library(miceadds)
library(lme4)
library(ggplot2)
library(rms)
library(mgcv)
library(tidymv)
library("ggthemes")
library(broom)
library(erer)
library(stringr)
library(tidyverse)

Loading required package: ggplot2


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘mice’


The following objects are masked from ‘package:base’:

    cbind, rbind


* miceadds 3.10-28 (2020-07-29 21:56:24)

Loading required package: Matrix

Registered S3 methods overwritten by 'lme4':
  method                          from
  cooks.distance.influence.merMod car 
  influence.merMod                car 
  dfbeta.influence.merMod         car 
  dfbetas.influence.merMod        car 

Loading required package: Hmisc

Loading required package: lattice

Loading required package: survival

Loading required package: Formula


Attaching package: ‘Hmisc’


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


Loading required p

In [2]:
#Read the complete set (no imputation)
UKBB_AG2 <- read.delim("~/jupyter/UKBB_AG2_07NOV20.txt", header = TRUE, na.strings=c("",".","NA")) %>% select(f.eid,T2D_status,GRS_WT_LIR,GRS_WT_IR,GRS_RAW_T2DIR,
                                 GRS_RAW_T2DIR2d2,GRS_WT_IR2d2,GRS_RAW_LIR2d2,GRS_WT_L5E8IR,GRS_RAW_LIR,GRS_RAW_IR,
                                 GRS_WT_L5E8IRd,GRS_WT_L5E8IR2d2,GRS_WT_L1E5IR,GRS_WT_L1E5IRd,GRS_WT_L1E5IR2d2,ALBUMINERIA.0.0,
                                 GRS_RAW_T2DIRd,GRS_RAW_IR2d,GRS_WT_IR2d,GRS_RAW_LIR2d,GRS_WT_T2DIRd, 
                                 ESKD.0.0,CKD.0.0,DN.0.0,ALL.0.0,NONESKD.0.0,DNCKD.0.0,
                                 CTRL_DNCKD.0.0,ACR.0.0,EGFR.0.0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,SEX.0.0,
                                        IDEAL_DIET2.0.0,LIFESCORE,AGE.0.0,
                                        SES_TDI.0.0,BMI.0.0,EDUYEARS,SBP.0.0,
                                        GRS_RAW_LIR2q,GRS_RAW_IR2q,GRS_WT_IR2q,GRS_RAW_T2DIR2q,GRS_WT_L5E8IR2q,
                                        GRS_WT_L1E5IR2q,SMOKE.0.0)

#Dichotomize Outcomes for Logistic Regression
#1_CKD
UKBB_AG2$CKD_only.0.0 <- factor(ifelse(UKBB_AG2$CKD.0.0=="CKD controls","CKD controls",
                              ifelse(UKBB_AG2$CKD.0.0=="CKD","CKD",NA)),
                levels = c("CKD controls", "CKD"))
#Set the refernece
UKBB_AG2$CKD_only.0.0 <- relevel(UKBB_AG2$CKD_only.0.0, ref = "CKD controls")

#2_CKD Extreme
UKBB_AG2$CKD_ex.0.0 <- factor(ifelse(UKBB_AG2$CKD.0.0=="CKD controls","CKD controls",
                              ifelse(UKBB_AG2$CKD.0.0=="CKD extreme","CKD extreme",NA)),
                levels = c("CKD controls", "CKD extreme"))
#Set the refernece
UKBB_AG2$CKD_ex.0.0 <- relevel(UKBB_AG2$CKD_ex.0.0, ref = "CKD controls")

#3_Micro
UKBB_AG2$micro.0.0 <- factor(ifelse(UKBB_AG2$ALBUMINERIA.0.0=="micro","micro",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0=="normo","normo",NA)),
                levels = c("normo", "micro"))
#Set the reference
UKBB_AG2$micro.0.0 <- relevel(UKBB_AG2$micro.0.0, ref = "normo")

#4_Macro
UKBB_AG2$macro.0.0 <- factor(ifelse(UKBB_AG2$ALBUMINERIA.0.0=="macro","macro",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0=="normo","normo",NA)),
                levels = c("normo", "macro"))
#Set the reference
UKBB_AG2$macro.0.0 <- relevel(UKBB_AG2$macro.0.0, ref = "normo")

#5_Macro
UKBB_AG2$macro.0.0 <- factor(ifelse(UKBB_AG2$ALBUMINERIA.0.0=="macro","macro",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0=="normo","normo",NA)),
                levels = c("normo", "macro"))

#6_ESKD vs. Macro
UKBB_AG2$ESKD_macro.0.0 <- factor(ifelse(UKBB_AG2$ESKD.0.0=="yes","ESKD",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0=="macro","macro",NA)),
                levels = c("macro","ESKD"))

#7_DNCKD vs. Control DNCKD
UKBB_AG2$DNCKD2.0.0 <- factor(ifelse(UKBB_AG2$DNCKD.0.0=="yes","DNCKD",
                              ifelse(UKBB_AG2$CTRL_DNCKD.0.0=="yes","DNCKD Control",NA)),
                levels = c("DNCKD Control","DNCKD"))

#8_ESKD vs. Normo, Macro, Micro
UKBB_AG2$ESKD_Albu.0.0 <- factor(ifelse(UKBB_AG2$ESKD.0.0=="yes","ESKD",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0 %in% c("normo","macro","micro"),"albu",NA)),
                levels = c("albu","ESKD"))

#Set the reference
UKBB_AG2$macro.0.0 <- relevel(UKBB_AG2$macro.0.0, ref = "normo")

#Summarize Counts of Disease Outcomes
table(UKBB_AG2$CKD_only.0.0) #1
table(UKBB_AG2$CKD_ex.0.0) #2
table(UKBB_AG2$micro.0.0) #3
table(UKBB_AG2$macro.0.0) #4
table(UKBB_AG2$ESKD.0.0) #5
table(UKBB_AG2$DN.0.0) #6
table(UKBB_AG2$ALL.0.0) #7
table(UKBB_AG2$ESKD.0.0) #8
table(UKBB_AG2$ESKD_macro.0.0) #8
table(UKBB_AG2$ESKD_Albu.0.0) #9
table(UKBB_AG2$DNCKD2.0.0) #10

#table smoking
table(UKBB_AG2$'SMOKE.0.0')
UKBB_AG2 <- UKBB_AG2 %>% filter(SMOKE.0.0!='Prefer not to answer')
table(UKBB_AG2$'SMOKE.0.0')


CKD controls          CKD 
      349669         6108 


CKD controls  CKD extreme 
      349669          984 


 normo  micro 
348496  14070 


 normo  macro 
348496   1120 


    no    yes 
356332    447 

...


macro  ESKD 
  963   447 


  albu   ESKD 
346560    447 


DNCKD Control         DNCKD 
       326513           645 


             Current                Never Prefer not to answer 
               38166               201909                 1269 
            Previous 
              132143 


             Current                Never Prefer not to answer 
               38166               201909                    0 
            Previous 
              132143 

In [3]:
table(UKBB_AG2$GRS_WT_IR2d2)


  high risk    low risk medium risk 
      37131       37555      297531 

In [4]:
#Test of interaction using continuous form of the ter
#FIXX REFERENCE TO T2D and ND
RESULTS_CONT <- data.frame()
RESULTS_OR <- data.frame()

for (ii in c('relevel(as.factor(SMOKE.0.0),"Never")*relevel(as.factor(GRS_RAW_T2DIR2d2),"low risk")',
             'relevel(as.factor(SMOKE.0.0),"Never")*relevel(as.factor(GRS_WT_IR2d2),"low risk")',
             'relevel(as.factor(SMOKE.0.0),"Never")*relevel(as.factor(GRS_RAW_LIR2d2),"low risk")',
             'relevel(as.factor(SMOKE.0.0),"Never")*relevel(as.factor(GRS_WT_L1E5IR2d2),"low risk")',
             'relevel(as.factor(SMOKE.0.0),"Never")*relevel(as.factor(GRS_WT_L5E8IR2d2),"low risk")')) {
    for(kk in c('Model1')){
          for (jj in c('relevel(as.factor(CKD_only.0.0),"CKD controls")','relevel(as.factor(CKD_ex.0.0),"CKD controls")',
                       'relevel(as.factor(micro.0.0),"normo")','relevel(as.factor(macro.0.0),"normo")',
                       'relevel(as.factor(ESKD.0.0),"no")',
                       'relevel(as.factor(DN.0.0),"no")','relevel(as.factor(ALL.0.0),"no")',
                       'relevel(as.factor(ESKD_macro.0.0),"macro")',
                       'relevel(as.factor(DNCKD2.0.0),"DNCKD Control")',
                       'relevel(as.factor(ESKD_Albu.0.0),"albu")')) {   
    
        #Set the term based on the factor
        if (ii == 'relevel(as.factor(SMOKE.0.0),"Never")*relevel(as.factor(GRS_RAW_T2DIR2d2),"low risk")') {
            ii2 <- 'relevel(as.factor(SMOKE.0.0),"Never")*GRS_RAW_T2DIR'
        }
        if (ii == 'relevel(as.factor(SMOKE.0.0),"Never")*relevel(as.factor(GRS_WT_IR2d2),"low risk")') {
            ii2 <- 'relevel(as.factor(SMOKE.0.0),"Never")*GRS_RAW_IR'
        } 
        if (ii == 'relevel(as.factor(SMOKE.0.0),"Never")*relevel(as.factor(GRS_RAW_LIR2d2),"low risk")') {
            ii2 <- 'relevel(as.factor(SMOKE.0.0),"Never")*GRS_RAW_LIR'
        }
        if (ii == 'relevel(as.factor(SMOKE.0.0),"Never")*relevel(as.factor(GRS_WT_L1E5IR2d2),"low risk")') {
            ii2 <- 'relevel(as.factor(SMOKE.0.0),"Never")*GRS_WT_L1E5IR'
        }
        if (ii == 'relevel(as.factor(SMOKE.0.0),"Never")*relevel(as.factor(GRS_WT_L5E8IR2d2),"low risk")') {
            ii2 <- 'relevel(as.factor(SMOKE.0.0),"Never")*GRS_WT_L1E5IR'
        }
              
        #Model 1 - Age, gender, PCI
        if(kk == "Model1"){
            
        fmla <- as.formula(paste0(jj," ~ ",ii, "+ AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10"), env = environment())
        fmla2 <- as.formula(paste0(jj," ~ ",ii2, "+ AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10"), env = environment())}
        if(kk == "Model2"){
            
        fmla <- as.formula(paste0(jj," ~ ", ii,"+ AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 +
                                        + EDUYEARS + SES_TDI.0.0"), env = environment())
        fmla2 <- as.formula(paste0(jj," ~ ", ii2,"+ AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 +
                                        + EDUYEARS + SES_TDI.0.0"), env = environment())}
          
        if(kk == "Model3"){
            
        fmla <- as.formula(paste0(jj," ~ ", ii,"+ AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 +
                                        + EDUYEARS + SES_TDI.0.0 + BMI.0.0"), env = environment())
        fmla2 <- as.formula(paste0(jj," ~ ", ii2,"+ AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 +
                                        + EDUYEARS + SES_TDI.0.0 + BMI.0.0"), env = environment())}
          
        M1 <- glm(fmla, data=UKBB_AG2, binomial(link="logit"))  
        M1b <- glm(fmla2, data=UKBB_AG2, binomial(link="logit")) 
        M1_2 <-  M1 %>% summary()
        M1_3 <- anova(M1b,test="LRT")
        
        #Model 1 - Sub-groups
        #Type 2 Diabetics
        M1_T2D <- UKBB_AG2 %>% filter(T2D_status == 1) %>% glm(formula=fmla, family=binomial(link="logit"))    
        M1_T2Db <- UKBB_AG2 %>% filter(T2D_status == 1) %>% glm(formula=fmla2, family=binomial(link="logit"))
        M1_T2D2 <- M1_T2D  %>% summary()
        M1_T2D3 <- anova(M1_T2Db ,test="LRT")
            
        #Non-Diabetes
        M1_ND <- UKBB_AG2 %>% filter(T2D_status == 0) %>% glm(formula=fmla,family=binomial(link="logit"))  
        M1_NDb <- UKBB_AG2 %>% filter(T2D_status == 0) %>% glm(formula=fmla2,family=binomial(link="logit"))
        M1_ND2 <- M1_ND  %>% summary()
        M1_ND3 <- anova(M1_NDb ,test="LRT")
        
        TEMP<- list(model=kk, var=jj,var2=ii, total=M1_2,t2d=M1_T2D2,nd=M1_ND2,lrt_tot=M1_3,
                    lrt_t2d=M1_T2D3,lrt_nd=M1_ND3)
                     
        #Confidence Intervals
        #Entire Sample
        M1$dwn_conf <- exp(coefficients(TEMP$total)[,1]-1.96*coefficients(TEMP$total)[,2])
        M1$up_conf <- exp(coefficients(TEMP$total)[,1]+1.96*coefficients(TEMP$total)[,2])
        TABLE2 <- round(cbind(OR=exp(coefficients(TEMP$total)[,1]),CI=cbind(M1$dwn_conf,M1$up_conf)),3)   
        #Diabetics
        M1_T2D$dwn_conf <- exp(coefficients(TEMP$t2d)[,1]-1.96*coefficients(TEMP$t2d)[,2])
        M1_T2D$up_conf <- exp(coefficients(TEMP$t2d)[,1]+1.96*coefficients(TEMP$t2d)[,2])
        TABLE2_T2D <- round(cbind(OR=exp(coefficients(TEMP$t2d)[,1]),CI=cbind(M1_T2D$dwn_conf,M1_T2D$up_conf)),3)
        #Non-Diabetics
        M1_ND2$dwn_conf <- exp(coefficients(TEMP$nd)[,1]-1.96*coefficients(TEMP$nd)[,2])
        M1_ND2$up_conf <- exp(coefficients(TEMP$nd)[,1]+1.96*coefficients(TEMP$nd)[,2])
        TABLE2_ND <- round(cbind(OR=exp(coefficients(TEMP$nd)[,1]),CI=cbind(M1_ND2$dwn_conf,M1_ND2$up_conf)),3)
            
        #Summary Statistics
        OR_CI <- list(model=kk, var=jj,var2=ii,nd_ci=TABLE2_ND,t2d_ci=TABLE2_T2D,all_ci=TABLE2)
        RESULTS_OR <- c(RESULTS_OR,OR_CI)    
        #Combined results
        RESULTS_CONT <- c(RESULTS_CONT,TEMP)
       }
    }
}

In [5]:
#Total Sample Score

#Intermediate and High Risk (Three Categories)
#Variable of interest listed first
#Only Save Two Coefficients

#6 = unique number of elements in list of 180 objects
#4,5,6 = model-based results (glm summary)
coef_all3 <- data.frame()
for (i in seq(from=1,to=length(RESULTS_CONT)/9,by=1)) {

  #identifiers
  model_id <- 2 + 9*(i-1) #model id
  i1 <- 4 + 9*(i-1) #total models
  i2 <- 5 + 9*(i-1) #t2d models
  i3 <- 6 + 9*(i-1) #nd models
    
  #lrt
  i7 <- 7 + 9*(i-1) #total models
  i8 <- 8 + 9*(i-1) #t2d models
  i9 <- 9 + 9*(i-1) #nd models 

    
  #model 1,2,3
  term <- i - (i%/%30)*30 + 1 #remainder from subtracting from multiple of 30
  i4 <- ifelse(term %in% c(1:10),
               'Model 1',
               ifelse(term %in% c(11:20),'Model 2',
                      ifelse(term %in% c(21:30),'Model 3')))
  
  #ENTIRE SAMPLE 
  #pull coefficients and convert to OR
  coef_all <- data.frame(round(exp(coefficients(RESULTS_CONT[[i1]])),2)[,1])
  id_model <- names(round(exp(coefficients(RESULTS_CONT[[i1]])),2)[,1])
  model_id2 <- RESULTS_CONT[[model_id]]
  rep_model <- rep(model_id2,dim(coef_all)[1])
  rep_adj <- rep(i4,dim(coef_all)[1])
  
  #pull coefficients and calculate 95% CI
  up_coef <- data.frame(round(exp(coefficients(RESULTS_CONT[[i1]])[,1]+1.96*coefficients(RESULTS_CONT[[i1]])[,2]),2))
  down_coef <- data.frame(round(exp(coefficients(RESULTS_CONT[[i1]])[,1]-1.96*coefficients(RESULTS_CONT[[i1]])[,2]),2))
  #likelihood ratio
  lrt_total <- round(RESULTS_CONT[[i7]][2,5],4)
  lrt_total2 <- rep(as.character(lrt_total),dim(coef_all)[1])
    
  coef_all2 <- cbind(rep_model,id_model) #model outcome to model var
  coef_all2 <- cbind(coef_all2,coef_all) #coefficients
  coef_all2 <- cbind(coef_all2,rep_adj)
  coef_all2 <- cbind(coef_all2,down_coef) #95 CI
  coef_all2 <- cbind(coef_all2,up_coef) #95 CI
  coef_all2 <- cbind(coef_all2,lrt_total2)
  coef_all3 <- rbind(coef_all3,coef_all2)
}

#reformat table
#rename
names(coef_all3) <- c("rep_model","id_model","OR","model_adj","lower","upper","lrt")
#combine HR and 95% CI
coef_all3$combo <- paste0(coef_all3$OR," (",coef_all3$lower,"-",coef_all3$upper,")")
coef_all3 <- subset(coef_all3, select = -c(3,5,6))
#high/low
coef_all3$category <- ifelse(str_detect(coef_all3$id_model,"high")==TRUE,"2_high","1_medium")
#substring
coef_all3$sub <- substr(coef_all3$id_model,19,32)
coef_all3$rep_model2 <- substr(coef_all3$rep_model,19,35)
#key
coef_all3$key <- paste0(coef_all3$sub,"-",coef_all3$rep_model2)
#spread
#coef_all3 <- subset(coef_all3, select = -c(1,2))


#T2D Sample Score

#Total Sample Score

#6 = unique number of elements in list of 180 objects
#4,5,6 = model-based results (glm summary)
coef_all3 <- data.frame()
for (i in seq(from=1,to=length(RESULTS_CONT)/9,by=1)) {

  #identifiers
  model_id <- 2 + 9*(i-1) #model id
  i1 <- 4 + 9*(i-1) #total models
  i2 <- 5 + 9*(i-1) #t2d models
  i3 <- 6 + 9*(i-1) #nd models
    
  #lrt
  i7 <- 7 + 9*(i-1) #total models
  i8 <- 8 + 9*(i-1) #t2d models
  i9 <- 9 + 9*(i-1) #nd models 
    
  #model 1,2,3
  term <- i - (i%/%30)*30 + 1 #remainder from subtracting from multiple of 30
  i4 <- ifelse(term %in% c(1:10),
               'Model 1',
               ifelse(term %in% c(11:20),'Model 2',
                      ifelse(term %in% c(21:30),'Model 3')))
  
  #ENTIRE SAMPLE 
  #pull coefficients and convert to OR
  coef_all <- data.frame(round(exp(coefficients(RESULTS_CONT[[i2]])),2)[,1])
  id_model <- names(round(exp(coefficients(RESULTS_CONT[[i2]])),2)[,1])
  model_id2 <- RESULTS_CONT[[model_id]]
  rep_model <- rep(model_id2,dim(coef_all)[1])
  rep_adj <- rep(i4,dim(coef_all)[1])
  
  #pull coefficients and calculate 95% CI
  up_coef <- data.frame(round(exp(coefficients(RESULTS_CONT[[i2]])[,1]+1.96*coefficients(RESULTS_CONT[[i2]])[,2]),2))
  down_coef <- data.frame(round(exp(coefficients(RESULTS_CONT[[i2]])[,1]-1.96*coefficients(RESULTS_CONT[[i2]])[,2]),2))
  #likelihood ratio
  lrt_total <- round(RESULTS_CONT[[i8]][2,5],3)
  lrt_total2 <- rep(as.character(lrt_total),dim(coef_all)[1])
    
  coef_all2 <- cbind(rep_model,id_model) #model outcome to model var
  coef_all2 <- cbind(coef_all2,coef_all) #coefficients
  coef_all2 <- cbind(coef_all2,rep_adj)
  coef_all2 <- cbind(coef_all2,down_coef) #95 CI
  coef_all2 <- cbind(coef_all2,up_coef) #95 CI
  coef_all2 <- cbind(coef_all2,lrt_total2)
  coef_all3 <- rbind(coef_all3,coef_all2)
}

#reformat table
#rename
names(coef_all3) <- c("rep_model","id_model","OR","model_adj","lower","upper","lrt")
#combine HR and 95% CI
coef_all3$combo <- paste0(coef_all3$OR," (",coef_all3$lower,"-",coef_all3$upper,")")
coef_all3 <- subset(coef_all3, select = -c(3,5,6))
#high/low
coef_all3$category <- ifelse(str_detect(coef_all3$id_model,"high")==TRUE,"2_high","1_medium")
#substring
coef_all3$sub <- substr(coef_all3$id_model,19,32)
coef_all3$rep_model2 <- substr(coef_all3$rep_model,19,35)
#key
coef_all3$key <- paste0(coef_all3$sub,"-",coef_all3$rep_model2)
#spread
#coef_all3 <- subset(coef_all3, select = -c(1,2))
coef_all3_t2d <- coef_all3

#Non-Diabetic Sample Score

#Total Sample Score

#6 = unique number of elements in list of 180 objects
#4,5,6 = model-based results (glm summary)
coef_all3 <- data.frame()
for (i in seq(from=1,to=length(RESULTS_CONT)/9,by=1)) {

  #identifiers
  model_id <- 2 + 9*(i-1) #model id
  i1 <- 4 + 9*(i-1) #total models
  i2 <- 5 + 9*(i-1) #t2d models
  i3 <- 6 + 9*(i-1) #nd models
    
  #lrt
  i7 <- 7 + 9*(i-1) #total models
  i8 <- 8 + 9*(i-1) #t2d models
  i9 <- 9 + 9*(i-1) #nd models 

    
  #model 1,2,3
  term <- i - (i%/%30)*30 + 1 #remainder from subtracting from multiple of 30
  i4 <- ifelse(term %in% c(1:10),
               'Model 1',
               ifelse(term %in% c(11:20),'Model 2',
                      ifelse(term %in% c(21:30),'Model 3')))
  
  #ENTIRE SAMPLE 
  #pull coefficients and convert to OR
  coef_all <- data.frame(round(exp(coefficients(RESULTS_CONT[[i3]])),2)[,1])
  id_model <- names(round(exp(coefficients(RESULTS_CONT[[i3]])),2)[,1])
  model_id2 <- RESULTS_CONT[[model_id]]
  rep_model <- rep(model_id2,dim(coef_all)[1])
  rep_adj <- rep(i4,dim(coef_all)[1])
  
  #pull coefficients and calculate 95% CI
  up_coef <- data.frame(round(exp(coefficients(RESULTS_CONT[[i3]])[,1]+1.96*coefficients(RESULTS_CONT[[i3]])[,2]),2))
  down_coef <- data.frame(round(exp(coefficients(RESULTS_CONT[[i3]])[,1]-1.96*coefficients(RESULTS_CONT[[i3]])[,2]),2))
  #likelihood ratio
  lrt_total <- round(RESULTS_CONT[[i9]][2,5],3)
  lrt_total2 <- rep(as.character(lrt_total),dim(coef_all)[1])
    
  coef_all2 <- cbind(rep_model,id_model) #model outcome to model var
  coef_all2 <- cbind(coef_all2,coef_all) #coefficients
  coef_all2 <- cbind(coef_all2,rep_adj)
  coef_all2 <- cbind(coef_all2,down_coef) #95 CI
  coef_all2 <- cbind(coef_all2,up_coef) #95 CI
  coef_all2 <- cbind(coef_all2,lrt_total2)
  coef_all3 <- rbind(coef_all3,coef_all2)
}

#reformat table
#rename
names(coef_all3) <- c("rep_model","id_model","OR","model_adj","lower","upper","lrt")
#combine HR and 95% CI
coef_all3$combo <- paste0(coef_all3$OR," (",coef_all3$lower,"-",coef_all3$upper,")")
coef_all3 <- subset(coef_all3, select = -c(3,5,6))
#high/low
coef_all3$category <- ifelse(str_detect(coef_all3$id_model,"high")==TRUE,"2_high","1_medium")
#substring
coef_all3$sub <- substr(coef_all3$id_model,19,32)
coef_all3$rep_model2 <- substr(coef_all3$rep_model,19,35)
#key
coef_all3$key <- paste0(coef_all3$sub,"-",coef_all3$rep_model2)
#spread
#coef_all3 <- subset(coef_all3, select = -c(1,2))
coef_all3_nd <- coef_all3

#Export the Sheet
write.csv2(coef_all3,'/cellar/users/agarduno/jupyter/Analysis/All_EMMSmoke_16Nov20.txt')
write.csv2(coef_all3_t2d,'/cellar/users/agarduno/jupyter/Analysis/T2D_EMMSmoke_16Nov20.txt')
write.csv2(coef_all3_nd,'/cellar/users/agarduno/jupyter/Analysis/ND_EMMSmoke_16Nov20.txt')